In [ ]:
# Developed by Chris Nedlik

In [ ]:
######################################################################
##### List files manually ############################################
######################################################################

files = ['']

In [ ]:

     ### OR ###  
    

In [ ]:
######################################################################
##### Load Files from a list of datasets #############################
######################################################################

# Use: >> ls -d 2019* > datalist.txt
# to create lists of datasets

reprocess = False # Set to true to reprocess data

fileslist = "Temp.txt"

files = []
import csv
with open(fileslist,"rU") as names:
        csvReader = csv.reader(names)
        for row in csvReader:
            files.extend([str(row[0])])


In [ ]:
areawindow_low = 5     # samples below peak max used in area measurement
areawindow_high = 13   # samples above peak max used in area measurement
baseline_samples = 12  # samples at beginning of event window used to measure properties of background noise
gain_correct = 7.0    # Multiplier for top PMT areas such that TBA is flat and symmetric 
eventwindow = 100      # number of samples in the event window

import time
t0 = time.time()
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
from matplotlib.colors import LogNorm
from pylab import rcParams
from os import path

def bitfield(n):
    tmp = [1 if digit=='1' else 0 for digit in bin(n)[2:]]
    return tmp[::-1]
def Read_DDC10_BinWaveCap_ChSel(fName):
    waveInfo = {}       
    fp = open(fName,"rb")    
    numEvents = int(np.fromfile(fp,dtype=np.uint32,count=1))
    waveInfo['numEvents'] = numEvents
    numSamples = int(np.fromfile(fp,dtype=np.uint32,count=1))
    waveInfo['numSamples'] = numSamples
    chSelMask = int(np.fromfile(fp,dtype=np.uint32,count=1))
    chMap = np.where(bitfield(chSelMask))[0]
    waveInfo['chMap'] = chMap
    numChannels = len(chMap)
    waveInfo['numChannels'] = numChannels
    byteOrderPattern = hex(int(np.fromfile(fp,dtype=np.uint32,count=1)))    
    waveArr = np.zeros((numChannels,numEvents,numSamples),dtype=np.int16)    
    for ievt in range(numEvents):
        for ich in range(numChannels):
            dummy = np.fromfile(fp,dtype=np.uint32,count=2)
            waveTmp = np.fromfile(fp,dtype=np.int16,count=numSamples)
            waveArr[ich,ievt,:] = waveTmp
            dummy = np.fromfile(fp,dtype=np.uint32,count=1)
    fp.close()
    return (waveArr,waveInfo)

CPUdt = 1000000000.0/600000000.0 # 600 MHz time steps in ns 

for filename in files:
    
    if path.exists(filename+"/"+filename+".npz") & !reprocess:
        print(filename+" has already been processed")
        
    else:

        areawindow = areawindow_high+areawindow_low
        eventnum = []; timestamp = []; livetime = []; deadtime = []; 
        baseline_0  = []; baseline_1  = []; datalen=[];

        waveArr, waveInfo = Read_DDC10_BinWaveCap_ChSel("/srv/smbshare/BindToHome/"+filename+"/"+filename+".bin")

        with open(filename+"/"+filename+".txt","rU") as data:
            csvReader = csv.reader(data)
            for row in csvReader:
                if len(row) > 2:
                    eventnum.extend([int(row[0])-1])
                    timestamp.extend([round((int(row[1])*CPUdt)/1000.0)])
                    livetime.extend([round((int(row[2])*CPUdt)/1000.0)])
                    deadtime.extend([round((int(row[3])*CPUdt)/1000.0)])

        for i in range(0, len(waveArr[0])):
            baseline_0.extend([np.mean(waveArr[0,i,0:baseline_samples])])
            baseline_1.extend([np.mean(waveArr[1,i,0:baseline_samples])])
            waveArr[0,i] = waveArr[0,i]-baseline_0[i]
            waveArr[1,i] = waveArr[1,i]-baseline_1[i]

        waveArrSum = np.array(np.sum((gain_correct*waveArr[0,:,:], waveArr[1,:,:]), axis=0))
        eventnum = np.array(eventnum)
        timestamp = np.array(timestamp)
        livetime = np.array(livetime)
        deadtime = np.array(deadtime)     
        ts = timestamp - timestamp[0]
        runtime = ts[len(ts)-1]/1000000.0

        amp_0a  = []; amp_0b  = []; amp_1a  = []; amp_1b  = []; amp_Sa  = []; amp_Sb  = [];
        area_0a = []; area_0b = []; area_1a = []; area_1b = []; area_Sa = []; area_Sb = [];
        indx_0a = []; indx_0b = []; indx_1a = []; indx_1b = []; indx_Sa = []; indx_Sb = [];
        post_peak_frac_0a = []; pre_peak_frac_0a = [];
        post_peak_frac_1a = []; pre_peak_frac_1a = [];
        post_peak_frac_Sa = []; pre_peak_frac_Sa = [];
        pulse_width_0a    = []; pulse_width_1a   = [];
        pulse_width_Sa    = []; pulse_width_Sa   = [];
        bl_stddev_0       = []; bl_stddev_1      = []; tba     = [];

        for i in range(0, len(waveArr[0])):

            bl_stddev_0.extend([np.std(waveArr[0,i,0:baseline_samples])])
            bl_stddev_1.extend([np.std(waveArr[1,i,0:baseline_samples])])

            # Primary Pulse Finder
            mx0, indx0 = max((waveArr[0,i,k], k) for k in range(0, len(waveArr[0,i])))
            mx1, indx1 = max((waveArr[1,i,k], k) for k in range(0, len(waveArr[1,i])))
            mx00, indx00 = max((waveArrSum[i,k], k) for k in range(0, len(waveArrSum[i])))

            indx_0a.extend([indx0]); indx_1a.extend([indx1]); indx_Sa.extend([indx00])
            amp_0a.extend([mx0]); amp_1a.extend([mx1]); amp_Sa.extend([mx00])

            area_0a.extend([sum(waveArr[0, i, indx0-areawindow_low:indx0+areawindow_high])*10]) # 10 ns sample width
            area_1a.extend([sum(waveArr[1, i, indx1-areawindow_low:indx1+areawindow_high])*10]) # 10 ns sample width  
            area_Sa.extend([sum(waveArrSum[i,indx00-areawindow_low:indx00+areawindow_high])*10])

            if area_0a[i]>0:
                post_peak_frac_0a.extend([float(sum(waveArr[0, i, indx0:indx0+areawindow_high])*10)/float(area_0a[i])])
                pre_peak_frac_0a.extend([float(sum(waveArr[0, i, indx0-areawindow_low:indx0])*10)/float(area_0a[i])])     
                S1_0 = float(area_0a[i]) 
                tmp_area = 0; counter = 0
                for z in range(indx0-areawindow_low,indx0+areawindow_high):
                    tmp_area = tmp_area + (waveArr[0, i, z]*10)
                    if ((tmp_area >= 0.10*S1_0) & (counter == 0)):
                        tenpercent = z
                        counter = counter + 1
                    if tmp_area >= 0.90*S1_0:
                        ninetypercent = z
                        break
                pulse_width_0a.extend([ninetypercent - tenpercent])
            else:
                post_peak_frac_0a.extend([-1]); pre_peak_frac_0a.extend([-1]); pulse_width_0a.extend([-1])

            if area_1a[i]>0:        
                post_peak_frac_1a.extend([float(sum(waveArr[1, i, indx0:indx0+areawindow_high])*10)/float(area_1a[i])])
                pre_peak_frac_1a.extend([float(sum(waveArr[1, i, indx0-areawindow_low:indx0])*10)/float(area_1a[i])])

                S1_1 = float(area_1a[i]) 
                tmp_area = 0; counter = 0
                for z in range(indx1-areawindow_low,indx1+areawindow_high):
                    tmp_area = tmp_area + (waveArr[1, i, z]*10)
                    if ((tmp_area >= 0.10*S1_1) & (counter == 0)):
                        tenpercent = z
                        counter = counter + 1
                    if tmp_area >= 0.90*S1_1:
                        ninetypercent = z
                        break
                pulse_width_1a.extend([ninetypercent - tenpercent])
            else:
                post_peak_frac_1a.extend([-1]); pre_peak_frac_1a.extend([-1]); pulse_width_1a.extend([-1])

            if area_Sa[i]>0:        
                post_peak_frac_Sa.extend([float(sum(waveArrSum[i, indx00:indx00+areawindow_high])*10)/float(area_Sa[i])])
                pre_peak_frac_Sa.extend([float(sum(waveArrSum[i, indx00-areawindow_low:indx00])*10)/float(area_Sa[i])])
                S1_S = float(area_Sa[i]) 
                tmp_area = 0; counter = 0
                for z in range(indx00-areawindow_low,indx00+areawindow_high):
                    tmp_area = tmp_area + (waveArrSum[i, z]*10)
                    if ((tmp_area >= 0.10*S1_S) & (counter == 0)):
                        tenpercent = z
                        counter = counter + 1
                    if tmp_area >= 0.90*S1_S:
                        ninetypercent = z
                        break
                pulse_width_Sa.extend([ninetypercent - tenpercent])
            else:
                post_peak_frac_Sa.extend([-1]); pre_peak_frac_Sa.extend([-1]); pulse_width_Sa.extend([-1])

            # 2nd S1 pulse finder
            if (indx0 < (eventwindow - areawindow_high)):
                mx2, indx2 = max((waveArr[0,i,l], l) for l in range(indx0+areawindow_high, len(waveArr[0,i])))
                indx_0b.extend([indx2]); amp_0b.extend([mx2])
                if (indx2 < (eventwindow - areawindow_high)):
                    area_0b.extend([sum(waveArr[0, i, indx2-areawindow_low:indx2+areawindow_high])*10]) 
                else:
                    area_0b.extend([sum(waveArr[0, i, indx2-areawindow_low:len(waveArr[1,i])])*10])
            else:
                area_0b.extend([0]); indx_0b.extend([0]); amp_0b.extend([0]);

            if (indx1 < (eventwindow - areawindow_high)):
                mx3, indx3 = max((waveArr[1,i,l], l) for l in range(indx1+areawindow_high, len(waveArr[1,i])))
                indx_1b.extend([indx3]); amp_1b.extend([mx3]);
                if (indx3 < (eventwindow - areawindow_high)):
                    area_1b.extend([sum(waveArr[0, i, indx3-areawindow_low:indx3+areawindow_high])*10])
                else:
                    area_1b.extend([sum(waveArr[1, i, indx3-areawindow_low:len(waveArr[1,i])])*10])   
            else:
                area_1b.extend([0]); indx_1b.extend([0]); amp_1b.extend([0]);

            if (indx00 < (eventwindow - areawindow_high)):
                mx01, indx01 = max((waveArrSum[i,l], l) for l in range(indx00+areawindow_high, len(waveArrSum[i])))
                indx_Sb.extend([indx01]); amp_Sb.extend([mx01])
                if(indx01 < (eventwindow - areawindow_high)):
                    area_Sb.extend([sum(waveArrSum[i, indx01-areawindow_low:indx01+areawindow_high])*10]) 
                else:
                    area_Sb.extend([sum(waveArrSum[i, indx01-areawindow_low:len(waveArrSum[i])])*10]) 
            else:
                area_Sb.extend([0]); indx_Sb.extend([0]); amp_Sb.extend([0]);

            if ((area_0a[i] > 0) & (area_1a[i] > 0)):
                tba.extend([(float((gain_correct*area_0a[i])-(area_1a[i]))/float((gain_correct*area_0a[i]+area_1a[i])))])
            else:
                tba.extend([-2])

        baseline_0   = np.array(baseline_0);      baseline_1   = np.array(baseline_1) 
        bl_stddev_0  = np.array(bl_stddev_0);     bl_stddev_1  = np.array(bl_stddev_1)
        amp_0a       = np.array(amp_0a);          amp_0b       = np.array(amp_0b) 
        amp_1a       = np.array(amp_1a);          amp_1b       = np.array(amp_1b) 
        amp_Sa       = np.array(amp_Sa);          amp_Sb       = np.array(amp_Sb) 
        area_0a      = np.array(area_0a);         area_0b      = np.array(area_0b)
        area_1a      = np.array(area_1a);         area_1b      = np.array(area_1b)
        area_Sa      = np.array(area_Sa);         area_Sb      = np.array(area_Sb)
        indx_0a      = np.array(indx_0a);         indx_0b      = np.array(indx_0b)
        indx_1a      = np.array(indx_1a);         indx_1b      = np.array(indx_1b)
        indx_Sa      = np.array(indx_Sa);         indx_Sb      = np.array(indx_Sb)
        separation_0 = np.array(indx_0b-indx_0a); separation_1 = np.array(indx_1b-indx_1a)
        separation_S = np.array(indx_Sb-indx_Sa)
        tba          = np.array(tba) 
        post_peak_frac_0a = np.array(post_peak_frac_0a); pre_peak_frac_0a = np.array(pre_peak_frac_0a);
        post_peak_frac_1a = np.array(post_peak_frac_1a); pre_peak_frac_1a = np.array(pre_peak_frac_1a);
        post_peak_frac_Sa = np.array(post_peak_frac_Sa); pre_peak_frac_Sa = np.array(pre_peak_frac_Sa);
        pulse_width_0a = np.array(pulse_width_0a);       pulse_width_1a = np.array(pulse_width_1a);
        pulse_width_Sa = np.array(pulse_width_Sa);
        datalen = int(len(area_0a))

        np.savez("/srv/smbshare/BindToHome/"+filename+"/"+filename+".npz",\
                 baseline_0 = baseline_0, \
                 baseline_1 = baseline_1, \
                 bl_stddev_0 = bl_stddev_0, \
                 bl_stddev_1 = bl_stddev_1, \
                 amp_0a = amp_0a, \
                 amp_0b = amp_0b, \
                 amp_1a = amp_1a, \
                 amp_1b = amp_1b, \
                 amp_Sa = amp_Sa, \
                 amp_Sb = amp_Sb, \
                 area_0a = area_0a, \
                 area_0b = area_0b, \
                 area_1a = area_1a, \
                 area_1b = area_1b, \
                 area_Sa = area_Sa, \
                 area_Sb = area_Sb, \
                 indx_0a = indx_0a, \
                 indx_0b = indx_0b, \
                 indx_1a = indx_1a, \
                 indx_1b = indx_1b, \
                 indx_Sa = indx_Sa, \
                 indx_Sb = indx_Sb, \
                 separation_0 = separation_0, \
                 separation_1 = separation_1, \
                 separation_S = separation_S, \
                 tba = tba, \
                 post_peak_frac_0a = post_peak_frac_0a, \
                 post_peak_frac_1a = post_peak_frac_1a, \
                 post_peak_frac_Sa = post_peak_frac_Sa, \
                 pre_peak_frac_0a = pre_peak_frac_0a, \
                 pre_peak_frac_1a = pre_peak_frac_1a, \
                 pre_peak_frac_Sa = pre_peak_frac_Sa, \
                 pulse_width_0a = pulse_width_0a, \
                 pulse_width_1a = pulse_width_1a, \
                 pulse_width_Sa = pulse_width_Sa, \
                 areawindow_low = areawindow_low, \
                 areawindow_high = areawindow_high, \
                 baseline_samples = baseline_samples, \
                 datalen = datalen, \
                 gain_correct = gain_correct, \
                 eventwindow = eventwindow, \
                 eventnum = eventnum, \
                 livetime = livetime, \
                 deadtime = deadtime, \
                 runtime = runtime, \
                 timestamp = timestamp, \
                 waveArr = waveArr, \
                 waveArrSum = waveArrSum)

        print("Finished Saving "+filename+".npz - Elapsed time = "+str(round(time.time()-t0, 2))+' seconds')
    
print('Final run time = '+str(round(time.time()-t0, 2))+' seconds')

In [ ]:
### Waveform Viewer ###

# Event number: 
events = [177]
x1 = 10
x2 = 40

plt.figure(figsize=(14,5))
for evt in events:
    plt.plot(waveArr[0, evt]*gain_correct, label=' Top PMT')
    plt.plot(waveArr[1, evt], label=' Bottom PMT')
    #plt.plot(waveArrSum[evt], label=str(evt)+' Sum')
plt.xlabel('Sample')
plt.ylabel('mV')
plt.title('Event '+str(evt))
#plt.yscale('log')
#plt.xlim(x1,x2)
plt.legend()
plt.grid()
plt.show()